In [ ]:
import numpy as np
import pickle

In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()

figures_path = Path(os.environ.get("FIGURES_DIR"))
tsunami_dir = Path(os.environ.get("TSUNAMI_DIR"))
figures_path, tsunami_dir

In [ ]:
eval_path = tsunami_dir / "gp_eval.pkl"
eval_data = pickle.load(open(eval_path, 'rb'))

In [ ]:
# output_folder = WORK_DIR + "/projects/icml-2024/experiments/tsunami_with_bathymetry/outputs/63119/"
# slice_path = output_folder + "slice_data.pkl"
# slice_data = pickle.load(open(slice_path, 'rb'))

In [ ]:
X = eval_data["X"]
means = eval_data["means"]
covs = eval_data["covs"]
Hs = -eval_data["Hs"] * 1000

covs = covs.reshape(3, -1)
covs = np.moveaxis(covs, 0, 1)
covs = covs.reshape(means.shape)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors
import cmocean

def plot_at_fixed_time(X, mean, cov, Hs, fig=None, ax=None, vmax=1.0, zlims=(-3, 1.5)):
    # bath_vals = bathymetry_vals(X[..., 1:])
    t = X[0, 0, 0]
    if fig is None or ax is None:
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection="3d")
    baths = Hs / 500
    divnorm=colors.TwoSlopeNorm(vmin=baths.min(), vcenter=0., vmax=Hs.max())
    ax.plot_surface(X[..., 1], X[..., 2], baths, cmap=cmocean.cm.topo, norm=divnorm, alpha=1.0)

    ax.plot_surface(
        X[..., 1], X[..., 2], mean[..., 0], alpha=0.8, cmap=cmocean.cm.dense
    )
    ax.plot_surface(
        X[..., 1], X[..., 2], mean[..., 0] + 1.96 * np.sqrt(cov[..., 0]), alpha=0.6, cmap=cmocean.cm.dense
    )
    ax.set_zlim(*zlims)
    ax.tick_params(axis="x", pad=-2)
    ax.tick_params(axis="y", pad=-2)
    ax.tick_params(axis="z", pad=0)
    ax.set_xlabel("x (km)", labelpad=-7)
    ax.set_ylabel("y (km)", labelpad=-7.8)
    ax.set_yticks([0, 30, 60, 100])
    ax.set_zlabel("h (m)", labelpad=-16.5)
    ax.set_zticks([-2, 0, 2])
    # # Rotate viewpoint
    ax.view_init(azim=-70, elev=25)
    dist_x = X[..., 1].max()
    dist_y = X[..., 2].max()
    ax.set_box_aspect([dist_x / dist_y, 1, 1])
    # ax.set_title(f"t = {t/60:.1f} mins")

In [ ]:
from tueplots import bundles
from tueplots import figsizes
plt.rcParams.update(bundles.neurips2024())
plt.rcParams.update(figsizes.neurips2024())

In [ ]:
k = 85
fig = plt.figure()
# ax = fig.add_subplot(221, projection="3d")
# plot_at_fixed_time(X[0], means[0], covs[0], Hs[0], vmax=1.96, fig=fig, ax=ax, zlims=(-3, 2))
# ax = fig.add_subplot(222, projection="3d")
# plot_at_fixed_time(X[k], means[k], covs[k], Hs[k], vmax=1.96, fig=fig, ax=ax, zlims=(-3, 0.5))
# ax = fig.add_subplot(223)
# ax.imshow(covs[0][..., 0], vmin=covs.min(), vmax=covs.max())
# ax = fig.add_subplot(224)
# ax.imshow(covs[k][..., 0], vmin=covs.min(), vmax=covs.max())

N_frames = 3
# indices = np.linspace(0, 120 - 1, N_frames, dtype=int)
indices = [0, 40, 100]
print(indices)

for i, label in zip(range(N_frames), ["a", "b", "c"]):
    ax = fig.add_subplot(1, N_frames, i+1, projection="3d")
    plot_at_fixed_time(X[indices[i]], means[indices[i]], covs[indices[i]], Hs[indices[i]], vmax=1.96, fig=fig, ax=ax, zlims=(-2, 2))
    title = "\\textbf{" + label + ")}$\,$ t = " + f"{X[indices[i], 0, 0, 0]/60:.1f} mins"
    ax.set_title(title)

fig.savefig(figures_path / "tsunami_plot.pdf")

In [ ]:
fig, ax = plt.subplots(1, 3)

stds = np.sqrt(covs)
ax[0].imshow(covs[0][..., 0].T, vmin=covs.min(), vmax=covs.max())
ax[0].set_xlabel("x index")
ax[0].set_ylabel("y index")
ax[0].set_title("\\textbf{a)} t = 0 mins")
ax[1].imshow(covs[indices[1]][..., 0].T, vmin=covs.min(), vmax=covs.max())
# ax[1].set_title("\\mathbf\{b\} t =  mins")
frame1_str = "\\textbf{b)} t = " + f"{X[indices[1], 0, 0, 0]/60:.1f} mins"
ax[1].set_title(frame1_str)
ax[1].set_xlabel("x index")
im2 = ax[2].imshow(covs[indices[2]][..., 0].T, vmin=covs.min(), vmax=covs.max())
frame2_str = "\\textbf{c)} t = " + f"{X[indices[2], 0, 0, 0]/60:.1f} mins"
ax[2].set_title(frame2_str)
ax[2].set_xlabel("x index")
cbar_ax = fig.add_axes([1.01, 0.15, 0.02, 0.7])  # Adjust the position and size as needed
cbar = fig.colorbar(im2, cax=cbar_ax)

fig.savefig(figures_path / "tsunami_cov.pdf")

In [ ]:
from matplotlib.animation import FuncAnimation
from tqdm.notebook import tqdm

FPS = 20

def animated_plot_batched(X, means, covs, Hs):
    fps = FPS
    interval = 1000 / fps
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

    def animate(i):
        ax.clear()
        plot_at_fixed_time(X[i], means[i], covs[i], Hs[i], fig=fig, ax=ax, vmax=1.96)

    anim = FuncAnimation(
        fig, animate, frames=tqdm(range(means.shape[0])), interval=interval
    )
    return anim

In [ ]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation, FFMpegWriter

plt.rcParams["animation.ffmpeg_path"] = WORK_DIR + "/ffmpeg-binary/ffmpeg"

anim = animated_plot_batched(X, means, covs, Hs)
FFwriter = FFMpegWriter(fps=FPS, extra_args=["-vcodec", "libx264"])
anim.save("./sendai.mp4", writer=FFwriter)